In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from matplotlib import image
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.config.list_physical_devices('GPU')

In [2]:
# A function to return a CNN model based on input hyperparameters
def createCNN(n_filters, filter_size, n_dense, num_conv_layers, input_shape, activation, n_output, dropout_conv = 0.0, dropout_dense = 0.0, batch_normalize = False, data_augment = False):
    model = models.Sequential()
    
    if data_augment:
        data_augmentation = tf.keras.Sequential([ 
                            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                            layers.experimental.preprocessing.RandomRotation(0.2)])
        model.add(data_augment)
    
    if batch_normalize:
        model.add(layers.BatchNormalization(input_size=input_size))
    for ii in range(0, num_conv_layers): # Number of convolutional layers is num_conv_layers
        if ii == 0:
            model.add(layers.Conv2D(n_filters, (filter_size, filter_size), activation=activation, input_shape=input_shape))
        else:
            model.add(layers.Conv2D(n_filters, (filter_size, filter_size), activation=activation))
        model.add(layers.MaxPooling2D((2, 2)))
        if dropout_conv != 0.0:
            model.add(layers.Dropout(dropout_conv))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(n_dense, activation=activation))
    if dropout_dense != 0.0:
        model.add(layers.Dropout(dropout_dense))
    model.add(layers.Dense(n_output))
    
    return model

In [ ]:
dataset_path = "../../nature_12K/inaturalist_12K"
train_path = dataset_path + "/train"
test_path = dataset_path + "/val"

classes = [u for u in os.listdir(train_path) if u != '.DS_Store']

max_shape = (800, 800, 3)

for each in classes:
    curr_path = train_path + "/" + each
    files = [u for u in os.listdir(curr_path) if u != '.DS_Store']
    max_width = 0
    max_height = 0
    for every in files:
        temp = np.asarray(image.imread(curr_path + "/" + every))
        temp = np.pad(temp, (((max_shape[0] - temp.shape[0]) // 2,(max_shape[0] - temp.shape[0] + 1) // 2), ((max_shape[1] - temp.shape[1]) // 2,(max_shape[1] - temp.shape[1] + 1) // 2) , (0, 0)), 'constant')
        print(temp.shape)
    break

In [10]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

max_shape = (32, 32, 3)

train_size = int(train_labels.shape[0] * 0.9)
val_size = int(train_labels.shape[0] * 0.1)
test_size = test_labels.shape[0]

data_train = ImageDataGenerator(validation_split = 0.1)

data_test = ImageDataGenerator()

train_generator = data_train.flow(
        train_images,
        train_labels,
        batch_size=train_size,
        subset = 'training')

validation_generator = data_train.flow(
        train_images,
        train_labels,
        batch_size=val_size,
        subset = 'validation')

test_generator = data_train.flow(
        test_images,
        test_labels,
        batch_size=test_size)

In [ ]:
my_model = my_cnn(16, 3, 64, (32, 32, 3), 'relu', 10)
my_model.summary()

In [ ]:
dataset_path = "../../nature_12K/inaturalist_12K"
train_path = dataset_path + "/train"
test_path = dataset_path + "/val"
max_shape = (800, 800, 3)

data_train = ImageDataGenerator(validation_split = 0.1)

data_test = ImageDataGenerator()

train_generator = data_train.flow_from_directory(
        train_path,
        target_size=(max_shape[0], max_shape[1]),
        batch_size=32,
        class_mode='binary',
        subset = 'training')

validation_generator = data_train.flow_from_directory(
        train_path,
        target_size=(max_shape[0], max_shape[1]),
        batch_size=32,
        class_mode='binary',
        subset = 'validation')

test_generator = data_train.flow_from_directory(
        test_path,
        target_size=(max_shape[0], max_shape[1]),
        batch_size=32,
        class_mode='binary')

In [6]:
def train(model,
          optimizer,
          loss_fn,
          epochs=10):
  
    model.compile(optimizer=optimizer, loss=loss_fn, metrics = ['accuracy'])
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[WandbCallback()])

In [13]:
default_config = {
              "n_filters": 10,
              "filter_factor" : 1,
              "data_augment": False,
              "dropout" : 0.0,
              "batch_normalize" : False
           }

run = wandb.init(project='cs6910-a2', config=default_config)
config = wandb.config

# initialize model
model = createCNN(n_filters = config.n_filters, filter_size = 2, n_dense = 1, num_conv_layers = 3, input_shape = max_shape, activation = 'relu', n_output = 10, dropout_conv = config.dropout / 3, dropout_dense = config.dropout, batch_normalize=config.batch_normalize, data_augment=config.data_augment)

# Instantiate an optimizer to train the model.
optimizer = tf.keras.optimizers.Nadam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train(model,
      optimizer,
      loss_fn)

run.join()

wandb: wandb version 0.10.25 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[45000,10,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_4/conv2d_14/Relu (defined at C:\anaconda3\lib\site-packages\wandb\integration\keras\keras.py:119) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_37300]

Function call stack:
train_function


In [ ]:
import random

a = [[[0 for k in range(4)] for i in range(4)] for j in range(15)]
b = [0 for k in range(15)]

for i in range(0, 15):
    for j in range(0, 4):
        for k in range(0, 4):
            a[i][j][k] = random.randint(0, 15)
    b[i] = random.randint(1, 3)

f = open('train.txt', 'w')
f.write(str(a))
f.close()
f = open('test.txt', 'w')
f.write(str(b))
f.close()

In [ ]:
import ast

f = open('train.txt', 'r')
train_s = np.asarray(ast.literal_eval(f.read()))
print(train_s)
f.close()

f = open('test.txt', 'r')
test_s = np.asarray(ast.literal_eval(f.read()))
print(test_s)
f.close()

In [ ]:
testing_model = createCNN(10, 2, 2, 3, (32, 32, 3), 'relu', 10)
testing_model.summary()

In [ ]:
testing_model.compile(optimizer='nadam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = testing_model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))